In [11]:
from datasets import load_dataset
from transformers import RobertaTokenizer, RobertaModel
from dataclasses import dataclass

import torch
import torch.nn as nn
from torch.utils.data import 

import pickle

In [12]:
MODEL_NAME = "FacebookAI/roberta-base"

In [13]:
vqa = load_dataset("Gapes21/vqa2", split='train')

In [14]:
vqa = vqa.train_test_split(0.1)

In [15]:
vqa

DatasetDict({
    train: Dataset({
        features: ['image', 'question', 'answer'],
        num_rows: 98536
    })
    test: Dataset({
        features: ['image', 'question', 'answer'],
        num_rows: 10949
    })
})

In [16]:
le = pickle.load(open("labelEncoder.pkl", "rb"))

c:\programming\projects\pj_env\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.4.2 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [17]:
@dataclass
class TextCollator:
    tokenizer: RobertaTokenizer
    
    def tokenize_text(self, texts):
        encoded_text = self.tokenizer(
            text=texts,
            padding='longest',
            max_length=24,
            truncation=True,
            return_tensors='pt',
            return_token_type_ids=True,
            return_attention_mask=True,
        )
        return {
            "input_ids": encoded_text['input_ids'].squeeze(),
            "token_type_ids": encoded_text['token_type_ids'].squeeze(),
            "attention_mask": encoded_text['attention_mask'].squeeze(),
        }
    
    def __call__(self, raw_batch_dict):
        question_batch = raw_batch_dict['question'] if isinstance(raw_batch_dict, dict) else [i['question'] for i in raw_batch_dict]
        answer_batch = le.transform(raw_batch_dict['answer']) if isinstance(raw_batch_dict, dict) else [le.transform(i['answer']) for i in raw_batch_dict]

        return {
            **self.tokenize_text(question_batch),
            'labels': torch.tensor(answer_batch, dtype=torch.int64),
        }

In [18]:
class RobertaQA(nn.Module):
    def __init__(
            self,
            num_labels: int,
            roberta_name:str = 'roberta-base'
    ):
        super().__init__()
        self.num_labels = num_labels
        self.roberta_name = roberta_name

        self.roberta = RobertaModel.from_pretrained(self.roberta_name)

        self.hidden_dim = self.roberta.config.hidden_size

        self.self_attn = nn.MultiheadAttention(self.hidden_dim, 4, 0.1)

        self.classifier = nn.Linear(self.hidden_dim, self.num_labels)

    def forward(
            self,
            input_ids,
            attention_mask,
            token_type_ids,
            labels
    ):
        
        x = self.roberta(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        ).last_hidden_state

        x = self.self_attn(x, x, x)

        print(x.shape)

        return x
    

In [19]:
collator = TextCollator(RobertaTokenizer.from_pretrained(MODEL_NAME))
model = RobertaQA(len(le.classes_), MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

c:\programming\projects\pj_env\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Sai Madhavan G\.cache\huggingface\hub\models--FacebookAI--roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
train_loader = 

SyntaxError: invalid syntax (525519296.py, line 1)